In [17]:
import mlflow
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [18]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

[<Experiment: artifact_location='/home/juan/Documents/python/mlops/mlops-bootcamp/mlruns/2', creation_time=1723658273640, experiment_id='2', last_update_time=1723658273640, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='/home/juan/Documents/python/mlops/mlops-bootcamp/mlruns/1', creation_time=1723054783896, experiment_id='1', last_update_time=1723054783896, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='/home/juan/Documents/python/mlops/mlops-bootcamp/mlruns/0', creation_time=1723054783886, experiment_id='0', last_update_time=1723054783886, lifecycle_stage='active', name='Default', tags={}>]

In [19]:
client.create_experiment(name="my-cool-experiment")

MlflowException: Experiment(name=my-cool-experiment) already exists. Error: (raised as a result of Query-invoked autoflush; consider using a session.no_autoflush block if this flush is occurring prematurely)
(sqlite3.IntegrityError) UNIQUE constraint failed: experiments.name
[SQL: INSERT INTO experiments (name, artifact_location, lifecycle_stage, creation_time, last_update_time) VALUES (?, ?, ?, ?, ?)]
[parameters: ('my-cool-experiment', None, 'active', 1723658673207, 1723658673207)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [21]:
from mlflow.entities import ViewType

runs = client.search_runs(experiment_ids='1',
                          filter_string="",
                          run_view_type= ViewType.ACTIVE_ONLY,
                          max_results=5,
                          order_by=["metrics.rmse ASC"]
                          )

In [22]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 650f030c34fd4ad5881b92f3faaabdf2, rmse: 6.3016
run id: 73c8e6e034a64d69a572ce3545034d49, rmse: 6.3045
run id: d348a6483c584be58230cef4ba20e68b, rmse: 6.3104
run id: d7c163caf6a34a99a701d0985edfbc6c, rmse: 6.3115
run id: bc6bf844d0644bb19ba7dc3f4134f091, rmse: 6.3184


In [20]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)



In [23]:

run_id = "650f030c34fd4ad5881b92f3faaabdf2"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")



Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1723658776819, current_stage='None', description=None, last_updated_timestamp=1723658776819, name='nyc-taxi-regressor', run_id='650f030c34fd4ad5881b92f3faaabdf2', run_link=None, source='/home/juan/Documents/python/mlops/mlops-bootcamp/mlruns/1/650f030c34fd4ad5881b92f3faaabdf2/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [24]:
model_uri

'runs:/650f030c34fd4ad5881b92f3faaabdf2/model'

In [34]:

model_name="nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: Staging
version: 3, stage: None


/tmp/ipykernel_38039/722282794.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [36]:
model_version = 3
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_38039/1957332551.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1723658776819, current_stage='Staging', description=None, last_updated_timestamp=1723659098155, name='nyc-taxi-regressor', run_id='650f030c34fd4ad5881b92f3faaabdf2', run_link=None, source='/home/juan/Documents/python/mlops/mlops-bootcamp/mlruns/1/650f030c34fd4ad5881b92f3faaabdf2/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [37]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1723658776819, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2024-08-14', last_updated_timestamp=1723659401622, name='nyc-taxi-regressor', run_id='650f030c34fd4ad5881b92f3faaabdf2', run_link=None, source='/home/juan/Documents/python/mlops/mlops-bootcamp/mlruns/1/650f030c34fd4ad5881b92f3faaabdf2/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [40]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [41]:
df = read_dataframe("/home/juan/Documents/python/mlops/mlops-bootcamp/data/green_tripdata_2021-01.parquet")

In [45]:
new_run_id="bc6bf844d0644bb19ba7dc3f4134f091"

In [46]:
client.download_artifacts(run_id=new_run_id, path='preprocessor', dst_path='.')

'/home/juan/Documents/python/mlops/mlops-bootcamp/preprocessor'

In [47]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [48]:
X_test = preprocess(df, dv)

target = "duration"
y_test = df[target].values

In [53]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

CPU times: user 81.6 ms, sys: 0 ns, total: 81.6 ms
Wall time: 80.9 ms


/home/juan/miniconda3/envs/mlops_venv/lib/python3.9/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])
/home/juan/miniconda3/envs/mlops_venv/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': 5.878874721516509}